# Pipelines

In [20]:
# pandasとtrain_test_splitを呼び出し
import pandas as pd
from sklearn.model_selection import train_test_split
# データ読み込み
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv( 'test.csv', index_col='Id')

# 'SalePrice'のない行を削除
# inplace指定で元のdata frameを置き換える
# inplaceでXそのものを書き換える
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
# 目的変数と説明変数に分割
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# 訓練データと検証データに分ける
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)

# 各列の入力データの種類を表示
print(X_train_full.dtypes)
# ユニークが10未満のカテゴリカル変数を出力
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]
# 数値のカラムも出力
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

## 必要な列のみを出力
# ユニーク10以下のカテゴリカル変数＋数値変数
my_cols = categorical_cols + numerical_cols
# それぞれ入力用データからコピー
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object


In [21]:
X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,...,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,...,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,843,468,81,0,0,0,0,0,1,2006


In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  #欠損値代入のためのSimpleImputer
from sklearn.preprocessing import OneHotEncoder  #カテゴリカル変数処理ようのOneHotEncoder
from sklearn.ensemble import RandomForestRegressor  #ランダムフォレスト
from sklearn.metrics import mean_absolute_error  #評価用のMAE

# 数値変数への準備：定数で欠損値を埋める
numerical_transformer = SimpleImputer(strategy='constant')

# カテゴリカル変数への準備
categorical_transformer = Pipeline(steps=[('Imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
# transformerの定義
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols), 
    ('cat', categorical_transformer, categorical_cols)]
)
# estimaterを定義
model = RandomForestRegressor(n_estimators=100, random_state=0)

# transformerとestimaterを指定：(名前, 関数)のタプルで定義
clf = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
# 訓練データでfit
clf.fit(X_train, y_train)
# 検証データで予測
preds = clf.predict(X_valid)
# MAEを計算
print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17861.780102739725


In [24]:
from sklearn import set_config
set_config(display='diagram')   
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBa...
                                                   'LotConfig', 'LandSlope',
                                                   'Condition1', 'Condition2',
                                                   'BldgType', 'HouseStyle',
                                                   'RoofStyle', 'RoofMatl',
                                                   'MasVnrType', 'ExterQual',
                                                   'ExterCond', 'Foundation',
                                                   'BsmtQual', 'BsmtCond',
                                                   'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...])])),
                ('model', RandomForestRegressor(random_state=0))])